# Problem 3

Use this notebook to write your code for problem 3.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 3D - Convolutional network

As in problem 2, we have conveniently provided for your use code that loads and preprocesses the MNIST data.

In [3]:
# load MNIST data into PyTorch format
import torch
import torchvision
import torchvision.transforms as transforms

# set batch size
batch_size = 32

# load training data downloaded into data/ folder
mnist_training_data = torchvision.datasets.MNIST('data/', train=True, download=True,
                                                transform=transforms.ToTensor())
# transforms.ToTensor() converts batch of images to 4-D tensor and normalizes 0-255 to 0-1.0
training_data_loader = torch.utils.data.DataLoader(mnist_training_data,
                                                  batch_size=batch_size,
                                                  shuffle=True)

# load test data
mnist_test_data = torchvision.datasets.MNIST('data/', train=False, download=True,
                                                transform=transforms.ToTensor())
test_data_loader = torch.utils.data.DataLoader(mnist_test_data,
                                                  batch_size=batch_size,
                                                  shuffle=False)

In [4]:
# look at the number of batches per epoch for training and validation
print(f'{len(training_data_loader)} training batches')
print(f'{len(training_data_loader) * batch_size} training samples')
print(f'{len(test_data_loader)} validation batches')

1875 training batches
60000 training samples
313 validation batches


In [5]:
# my model
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(1, 8, kernel_size=(3,3)),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.25),
    
    nn.Conv2d(8, 8, kernel_size=(3,3)),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.25),
    
    nn.Flatten(),
    nn.Linear(25*8, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
    # PyTorch implementation of cross-entropy loss includes softmax layer
)

In [6]:
# why don't we take a look at the shape of the weights for each layer
for p in model.parameters():
    print(p.data.shape)

torch.Size([8, 1, 3, 3])
torch.Size([8])
torch.Size([8, 8, 3, 3])
torch.Size([8])
torch.Size([64, 200])
torch.Size([64])
torch.Size([10, 64])
torch.Size([10])


In [7]:
# our model has some # of parameters:
count = 0
for p in model.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
print(f'total params: {count}')

total params: 14178


In [8]:
# For a multi-class classification problem
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters())

In [9]:
# Train the model for 10 epochs, iterating on the data in batches
n_epochs = 10

# store metrics
training_accuracy_history = np.zeros([n_epochs, 1])
training_loss_history = np.zeros([n_epochs, 1])
validation_accuracy_history = np.zeros([n_epochs, 1])
validation_loss_history = np.zeros([n_epochs, 1])

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/10:', end='')
    train_total = 0
    train_correct = 0
    # train
    model.train()
    for i, data in enumerate(training_data_loader):
        images, labels = data
        optimizer.zero_grad()
        # forward pass
        output = model(images)
        # calculate categorical cross entropy loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        optimizer.step()
        
        # track training accuracy
        _, predicted = torch.max(output.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        # track training loss
        training_loss_history[epoch] += loss.item()
        # progress update after 180 batches (~1/10 epoch for batch size 32)
        if i % 180 == 0: print('.',end='')
    training_loss_history[epoch] /= len(training_data_loader)
    training_accuracy_history[epoch] = train_correct / train_total
    print(f'\n\tloss: {training_loss_history[epoch,0]:0.4f}, acc: {training_accuracy_history[epoch,0]:0.4f}',end='')
        
    # validate
    test_total = 0
    test_correct = 0
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(test_data_loader):
            images, labels = data
            # forward pass
            output = model(images)
            # find accuracy
            _, predicted = torch.max(output.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            # find loss
            loss = criterion(output, labels)
            validation_loss_history[epoch] += loss.item()
        validation_loss_history[epoch] /= len(test_data_loader)
        validation_accuracy_history[epoch] = test_correct / test_total
    print(f', val loss: {validation_loss_history[epoch,0]:0.4f}, val acc: {validation_accuracy_history[epoch,0]:0.4f}')

Epoch 1/10:...........
	loss: 0.4470, acc: 0.8573, val loss: 0.1504, val acc: 0.9588
Epoch 2/10:...........
	loss: 0.2718, acc: 0.9134, val loss: 0.1142, val acc: 0.9677
Epoch 3/10:...........
	loss: 0.2541, acc: 0.9192, val loss: 0.1451, val acc: 0.9543
Epoch 4/10:...........
	loss: 0.2455, acc: 0.9230, val loss: 0.1130, val acc: 0.9653
Epoch 5/10:...........
	loss: 0.2490, acc: 0.9232, val loss: 0.0955, val acc: 0.9681
Epoch 6/10:...........
	loss: 0.2404, acc: 0.9257, val loss: 0.1085, val acc: 0.9667
Epoch 7/10:...........
	loss: 0.2417, acc: 0.9250, val loss: 0.1311, val acc: 0.9635
Epoch 8/10:...........
	loss: 0.2355, acc: 0.9271, val loss: 0.1154, val acc: 0.9636
Epoch 9/10:...........
	loss: 0.2349, acc: 0.9269, val loss: 0.0980, val acc: 0.9692
Epoch 10/10:...........
	loss: 0.2320, acc: 0.9287, val loss: 0.0879, val acc: 0.9732


Above, we output the training loss/accuracy as well as the validation loss and accuracy. Not bad! Let's see if you can do better.

In [52]:
ps = np.linspace(0.1, 1, 10)
n_ps = len(ps)
accuracies = []

training_accuracy_history = np.zeros([n_ps, 1])
training_loss_history = np.zeros([n_ps, 1])
validation_accuracy_history = np.zeros([n_ps, 1])
validation_loss_history = np.zeros([n_ps, 1])

for j, p in enumerate(ps):

    print(f'p = {p:.1f}:')
    model = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=(3,3)),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout(p=p),
        
        nn.Conv2d(16, 8, kernel_size=(3,3)),
        nn.BatchNorm2d(8),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout(p=p),

        nn.Flatten(),
        nn.Linear(25*8, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
        # PyTorch implementation of cross-entropy loss includes softmax layer
    )

    # For a multi-class classification problem
    import torch.optim as optim
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(model.parameters())

    train_total = 0
    train_correct = 0
    # train
    model.train()
    for i, data in enumerate(training_data_loader):
        images, labels = data
        optimizer.zero_grad()
        # forward pass
        output = model(images)
        # calculate categorical cross entropy loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        optimizer.step()
        
        # track training accuracy
        _, predicted = torch.max(output.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        # track training loss
        training_loss_history[epoch] += loss.item()
        # progress update after 180 batches (~1/10 epoch for batch size 32)
        if i % 180 == 0: print('.',end='')
    training_loss_history[j] /= len(training_data_loader)
    training_accuracy_history[j] = train_correct / train_total
    print(f'\n\tloss: {training_loss_history[j,0]:0.4f}, acc: {training_accuracy_history[j,0]:0.4f}',end='')
        
    # validate
    test_total = 0
    test_correct = 0
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(test_data_loader):
            images, labels = data
            # forward pass
            output = model(images)
            # find accuracy
            _, predicted = torch.max(output.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            # find loss
            loss = criterion(output, labels)
            validation_loss_history[epoch] += loss.item()
        validation_loss_history[j] /= len(test_data_loader)
        validation_accuracy_history[j] = test_correct / test_total
    print(f', val loss: {validation_loss_history[j,0]:0.4f}, val acc: {validation_accuracy_history[j,0]:0.4f}')



p = 0.1:
...........
	loss: 0.0000, acc: 0.9349, val loss: 0.0000, val acc: 0.9816
p = 0.2:
...........
	loss: 0.0000, acc: 0.9185, val loss: 0.0000, val acc: 0.9684
p = 0.3:
...........
	loss: 0.0000, acc: 0.9062, val loss: 0.0000, val acc: 0.9678
p = 0.4:
...........
	loss: 0.0000, acc: 0.8827, val loss: 0.0000, val acc: 0.9686
p = 0.5:
...........
	loss: 0.0000, acc: 0.8618, val loss: 0.0000, val acc: 0.9568
p = 0.6:
...........
	loss: 0.0000, acc: 0.7820, val loss: 0.0000, val acc: 0.9541
p = 0.7:
...........
	loss: 0.0000, acc: 0.7471, val loss: 0.0000, val acc: 0.9327
p = 0.8:
...........
	loss: 0.0000, acc: 0.5751, val loss: 0.0000, val acc: 0.8740
p = 0.9:
...........
	loss: 0.0000, acc: 0.3323, val loss: 0.0000, val acc: 0.8033
p = 1.0:
...........
	loss: 8.5142, acc: 0.1088, val loss: 154.1664, val acc: 0.0971


In [55]:
# final model
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=(3,3)),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.1),
        
    nn.Conv2d(16, 8, kernel_size=(3,3)),
    nn.BatchNorm2d(8),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.1),

    nn.Flatten(),
    nn.Linear(25*8, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
    # PyTorch implementation of cross-entropy loss includes softmax layer
  )

# our model has some # of parameters:
count = 0
for p in model.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
print(f'total params: {count}')

total params: 14882


In [56]:
# For a multi-class classification problem
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters())

# Train the model for 10 epochs, iterating on the data in batches
n_epochs = 10

# store metrics
training_accuracy_history = np.zeros([n_epochs, 1])
training_loss_history = np.zeros([n_epochs, 1])
validation_accuracy_history = np.zeros([n_epochs, 1])
validation_loss_history = np.zeros([n_epochs, 1])

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/10:', end='')
    train_total = 0
    train_correct = 0
    # train
    model.train()
    for i, data in enumerate(training_data_loader):
        images, labels = data
        optimizer.zero_grad()
        # forward pass
        output = model(images)
        # calculate categorical cross entropy loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        optimizer.step()
        
        # track training accuracy
        _, predicted = torch.max(output.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        # track training loss
        training_loss_history[epoch] += loss.item()
        # progress update after 180 batches (~1/10 epoch for batch size 32)
        if i % 180 == 0: print('.',end='')
    training_loss_history[epoch] /= len(training_data_loader)
    training_accuracy_history[epoch] = train_correct / train_total
    print(f'\n\tloss: {training_loss_history[epoch,0]:0.4f}, acc: {training_accuracy_history[epoch,0]:0.4f}',end='')
        
    # validate
    test_total = 0
    test_correct = 0
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(test_data_loader):
            images, labels = data
            # forward pass
            output = model(images)
            # find accuracy
            _, predicted = torch.max(output.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            # find loss
            loss = criterion(output, labels)
            validation_loss_history[epoch] += loss.item()
        validation_loss_history[epoch] /= len(test_data_loader)
        validation_accuracy_history[epoch] = test_correct / test_total
    print(f', val loss: {validation_loss_history[epoch,0]:0.4f}, val acc: {validation_accuracy_history[epoch,0]:0.4f}')

Epoch 1/10:...........
	loss: 0.2832, acc: 0.9136, val loss: 0.0986, val acc: 0.9696
Epoch 2/10:...........
	loss: 0.1126, acc: 0.9650, val loss: 0.0694, val acc: 0.9782
Epoch 3/10:...........
	loss: 0.0993, acc: 0.9701, val loss: 0.0635, val acc: 0.9780
Epoch 4/10:...........
	loss: 0.0874, acc: 0.9727, val loss: 0.0698, val acc: 0.9799
Epoch 5/10:...........
	loss: 0.0823, acc: 0.9744, val loss: 0.0560, val acc: 0.9812
Epoch 6/10:...........
	loss: 0.0763, acc: 0.9769, val loss: 0.0567, val acc: 0.9838
Epoch 7/10:...........
	loss: 0.0720, acc: 0.9778, val loss: 0.0534, val acc: 0.9838
Epoch 8/10:...........
	loss: 0.0715, acc: 0.9790, val loss: 0.0565, val acc: 0.9824
Epoch 9/10:...........
	loss: 0.0693, acc: 0.9792, val loss: 0.0496, val acc: 0.9845
Epoch 10/10:...........
	loss: 0.0686, acc: 0.9791, val loss: 0.0488, val acc: 0.9856
